In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016131877899169922
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 39.991386633334464
200000 69.29124467440622
300000 71.19786537978162
400000 69.21550300904339
500000 69.29486748482213
600000 72.83539387217604
700000 69.01188290706126
800000 68.45199569797468
900000 68.77024648566848
1000000 68.28539003318568
1100000 20.484537792984465
1200000 68.79887722175863
1300000 67.56943359176663
1400000 67.16232410319728
1500000 67.29503763044958
1600000 70.8544341578951
1700000 68.56030375319234
1800000 67.86964300093547
1900000 68.73076550803223
2000000 68.5888171267005
2100000 52.987391531776126
2200000 36.32226103257456
2300000 64.8278599753597
2400000 64.60221221776386
2500000 64.00732308982518
2600000 64.26016478275335
2700000 67.26685842305015
2800000 64.5807098592587
2900000 64.3964547812084
3000000 63.97029917220569
31

30100000 73.17309332379162
30200000 74.92025416931754
30300000 71.05363401599396
30400000 55.88581584708588
30500000 37.81278797342986
30600000 70.40980615953285
30700000 71.48939344937831
30800000 69.44119969654687
30900000 78.01517942034754
31000000 73.9250419602434
31100000 81.98171638896865
31200000 74.99408103928661
31300000 39.06845968567325
31400000 75.9906710967916
31500000 80.28871955809821
31600000 72.56723965341675
31700000 58.31391562251139
31800000 46.3305393629948
31900000 77.28947529940048
32000000 72.82149159558934
32100000 72.27038026839642
32200000 68.29311721709439
32300000 67.6541644972706
32400000 67.07470709988206
32500000 57.084404072708345
32600000 41.041488021221504
32700000 62.429513986341284
32800000 71.6655324564204
32900000 68.96082392547312
33000000 70.20797032246193
33100000 73.92114836880867
33200000 74.22115801367902
33300000 59.46250573542774
33400000 49.79704763939878
33500000 71.99430562288708
33600000 78.75283628162342
33700000 76.65776519013218
338

60600000 80.72834594575009
60700000 83.27392472226
60800000 76.45305705513692
60900000 75.78926923228694
61000000 74.6298248761334
61100000 70.82740495220091
61200000 72.2635752620344
61300000 52.807319482672625
61400000 48.5663485115065
61500000 62.45301289109155
61600000 63.47531964293393
61700000 68.95884151037798
61800000 68.6471226966725
61900000 30.06746953401335
62000000 61.94749170605689
62100000 66.58538102601449
62200000 65.63912682600689
62300000 69.16927819236665
62400000 68.67913751202333
62500000 33.88785232659842
62600000 74.33073929010887
62700000 77.72745727018469
62800000 71.25571977228881
62900000 79.11674221978426
63000000 80.33289515496593
63100000 74.77823827400216
63200000 81.29013060843235
63300000 77.49876834099787
63400000 60.793096715170236
63500000 44.99954044752895
63600000 70.14005340898875
63700000 70.24864491475384
63800000 69.62993431448851
63900000 72.89256016765415
64000000 29.725681316166586
64100000 67.72892992598575
64200000 69.47095455456444
64300

91000000 50.93131043197595
91100000 65.12279929528542
91200000 59.06084747309337
91300000 24.14594949855849
91400000 61.2881110484719
91500000 62.15336428670895
91600000 39.62752275191831
91700000 57.93727135859707
91800000 60.72538604925721
91900000 66.11878604159648
92000000 66.04837498217672
92100000 70.25636585108722
92200000 73.01263655683994
92300000 72.53360783448804
92400000 77.36351544486986
92500000 73.39167640990404
92600000 69.06574247842987
92700000 69.085826438453
92800000 69.88512401341544
92900000 68.30435875158267
93000000 49.78166728980819
93100000 48.43841379958774
93200000 66.90920577840157
93300000 62.923833251214354
93400000 26.229561078753303
93500000 65.35821107757982
93600000 69.75048810906392
93700000 31.819149225765116
93800000 68.4145239506496
93900000 68.51974677148802
94000000 63.026408372673636
94100000 68.66971900949837
94200000 69.69620906238434
94300000 74.5547212753326
94400000 73.41987623574526
94500000 66.26162405548271
94600000 69.76237582396737
94

120600000 30.487180696880653
120700000 35.689519684939924
120800000 43.17929495243649
120900000 60.0691340468861
121000000 60.672119645673575
121100000 62.508268011885065
121200000 63.406112877615634
121300000 66.34815335150958
121400000 61.804507238589274
121500000 62.12830105532999
121600000 66.03152035377552
121700000 69.96178229134964
121800000 65.76322971246778
121900000 66.1786569738418
122000000 62.47386344552429
122100000 61.35673953112058
122200000 61.21126939768495
122300000 62.266245056587394
122400000 61.57209769606918
122500000 64.89830434663209
122600000 42.77896637697768
122700000 21.808536045686832
122800000 25.95360705244296
122900000 52.44682650852703
123000000 65.61054360702121
123100000 60.816392986867406
123200000 63.280629756808395
123300000 63.29505713993415
123400000 64.00235422759724
123500000 62.81055963864259
123600000 66.65129841641513
123700000 69.27273274178752
123800000 72.39371802176956
123900000 62.23155271324062
124000000 58.61953936902551
124100000 56

...

799800000 0.23296262374922763
799900000 0.42194476730151576
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.2331141715964443
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.47651244486560956
800200000 1.6374747050697822
800300000 1.0107118167242228
800400000 0.9589415620450172
800500000 0.8927944939106406
800600000 0.8880763669126299
800700000 0.9001318606647677
800800000 1.0851284771746694
800900000 0.9459007768194232
801000000 0.8234019828967021
801100000 0.7517387713127215
801200000 0.8924956334225005
801300000 0.9715663335033893
801400000 0.9953142576039374
801500000 0.7696513765687784
801600000 0.7620788242118689
801700000 0.9274895808907946
80180000

813500000 0.8758999441396481
813600000 0.7951568865478509
813700000 0.8195203388346731
813800000 0.9516890482743271
813900000 0.849668307443481
814000000 0.8376033355290949
814100000 0.9051636090429273
814200000 0.882533068317375
814300000 0.8163857129043559
814400000 0.9026199128284269
814500000 0.8542318484816048
814600000 0.9342248599628727
814700000 0.891764413422452
814800000 0.8754185736527761
814900000 0.8981906490365504
815000000 0.8108737131862124
815100000 0.8225523109868924
815200000 0.8691691763214271
815300000 0.7478539072793133
815400000 0.8709980489871177
815500000 0.8344807575257083
815600000 0.7737748409977171
815700000 0.8674970246795555
815800000 0.7603905134957016
815900000 0.8438513877818313
816000000 1.3266218728326353
816100000 0.7756756892873129
816200000 0.8943395331147591
816300000 0.9809396191633417
816400000 0.682904820799003
816500000 0.9469050227064556
816600000 0.6959434610284746
816700000 0.8363636556033253
816800000 0.8434214328344444
816900000 0.875215

841200000 0.20909683294522763
841300000 0.16735118032609025
841400000 0.22909213745206794
841500000 0.16513335591883396
841600000 0.21149280084882777
841700000 0.22378804082590872
841800000 0.1448521446898222
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.2452960891942203
842000000 0.20398835954844094
842100000 0.1718463014054146
842200000 0.2392787110587835
842300000 0.16482844892159768
842400000 0.193262785312993
842500000 0.21121717525167466
842600000 0.15458732209648424
842700000 0.2162739924644669
842800000 0.17317942960815427
842900000 0.1689175755898482
843000000 0.2149112512016528
843100000 0.15343584733537832
843200000 0.19462906231906746
843300000 0.18564960215489137
843400000 0.14763200422931116
843500000 0.2165750197357946
843600000 0.1568607428006722
843700000 0.1890369144522786
843800000 0.1980974984683487
843900000 0.13973799657041033
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 1048576
844000000 0.20555507545161644
8441

861300000 0.033522372312634524
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03465022158792615
861500000 0.028722647831532028
861600000 0.016757205354170007
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.02574179220687151
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.019932971009143197
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.01958795312338273
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.01880333601194024
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.01560750555486679
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2, 2, 8, 4, 1] 128
cases of th

866100000 0.0006684903311530749
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.00020838250269293788
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so 